In [ ]:
import pandas as pd

In [ ]:
#This will list all files in the current directory, make sure the COVID files are in the current directory
ls

In [ ]:
df_may17 = pd.read_csv('COVID_Cases_Restricted_Detailed_05172020.csv')
df_may29 = pd.read_csv('COVID_Cases_Restricted_Detailed_05292020.csv')
df_june = pd.read_csv('COVID_Cases_Restricted_Detailed_Internal_06272020.csv')
df_july = pd.read_csv('COVID_Cases_Restricted_Detailed_07312020.csv')
df_aug = pd.read_csv('COVID_Cases_Restricted_Detailed_08312020.csv')
df_sep = pd.read_csv('COVID_Cases_Restricted_Detailed_09302020.csv')

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None): 
    print(df_july['cdc_report_dt'].value_counts())

In [ ]:
#df_june, df_july, df_aug and df_sep have a combined race_ethnicity column, lets break that out

df_june_split = pd.concat([df_june,df_june['race_ethnicity_combined'].str.split(', ', expand = True)], axis = 1)
df_june_split.rename(columns={0: 'race_combined', 1: 'ethnicity'}, inplace=True)

df_july_split = pd.concat([df_july,df_july['race_ethnicity_combined'].str.split(', ', expand = True)], axis = 1)
df_july_split.rename(columns={0: 'race_combined', 1: 'ethnicity'}, inplace=True)

df_aug_split = pd.concat([df_aug,df_aug['race_ethnicity_combined'].str.split(', ', expand = True)], axis = 1)
df_aug_split.rename(columns={0: 'race_combined', 1: 'ethnicity'}, inplace=True)

df_sep_split = pd.concat([df_sep,df_sep['race_ethnicity_combined'].str.split(', ', expand = True)], axis = 1)
df_sep_split.rename(columns={0: 'race_combined', 1: 'ethnicity'}, inplace=True)

#note at the far right new columns for race_combined and ethnicity
df_sep_split.head()

In [ ]:
#these are all the columns in the june (or later) dataframes that not in the two may dataframes
for x in df_june_split.columns:
    if x not in df_may17.columns or x not in df_may29.columns:
        print(x)

In [ ]:
#these are all the columns in the May (or later) dataframes that not in the June (or later) dataframes
for x in df_may17.columns:
    if x not in df_june_split.columns:
        print(x)
        
#nothing is printed (all the columns in the df_may17 dataframe are in the df_june_split dataframe)

In [ ]:
#just to check all the columns match in the June or later DFs
print(df_june_split.columns == df_july_split.columns)
print(df_july_split.columns == df_aug_split.columns)
print(df_aug_split.columns == df_sep_split.columns)
#it turns out the falses are due to different order not different columns (can use for loop like above to check)

In [ ]:
#make a list of data frames that get concated
all_dfs = [df_may17,df_may29,df_june_split,df_july_split,df_aug_split,df_sep_split]

working_df = pd.concat(all_dfs)
working_df.info()

In [ ]:
tr = len(df_may17) + len(df_may29) + len(df_june_split) + len(df_july_split) + len(df_aug_split) + len(df_sep_split)
tr1 = len(df_may17) + len(df_may29) + len(df_june) + len(df_july) + len(df_aug) + len(df_sep)

print(tr)
print(tr1)

In [ ]:
#taking a look at the data in the new working_df
working_df['race_combined'].value_counts()

In [ ]:
#Non-hispanic has trailing whitespace maybe?
#this probably won't be useful
working_df['ethnicity'].value_counts()

In [ ]:
working_df['sex'].value_counts()

In [ ]:
working_df['onset_dt'].dtype

In [ ]:
#Change some series data types to datetime
working_df['onset_dt'] = pd.to_datetime(working_df['onset_dt'], yearfirst = True)
working_df['pos_spec_dt'] = pd.to_datetime(working_df['pos_spec_dt'], yearfirst = True)
working_df['cdc_report_dt'] = pd.to_datetime(working_df['cdc_report_dt'], yearfirst = True, utc = True)
working_df.info()

In [ ]:
#it looks like most of the descriptive data (columns hosp_yn to medcond_yn is missing)
for x in range(8,29):
    print(working_df.iloc[:,x].value_counts())

In [ ]:
pd.options.display.float_format = '{:.2f}'.format
pd_pivot = pd.pivot_table(working_df, columns=['race_combined'], index=['res_state'], values ='cdc_report_dt', aggfunc = 'count')
pd_pivot['PercentUnknown'] = pd_pivot['Unknown']/pd_pivot.sum(axis=1)
pd_pivot['TotalCases'] = pd_pivot.sum(axis=1)
pd_pivot.sort_values(by=['TotalCases'], ascending = False)

In [ ]:
working_df['current_status'].value_counts()

In [ ]:
working_df_grouped = working_df.groupby(['race_combined','res_state']).agg({'cdc_report_dt':'count'})
with pd.option_context('display.max_rows', None, 'display.max_columns', None): 
    print(working_df_grouped)